第二章 end to end machine learning

一、结构化问题

二、获取数据与预处理

三、选择和训练模型

（一）在训练集上测试和评估

In [1]:
import os
os.environ["OMP_NUM_THREADS"]="7"
import pandas as pd
from sklearn.datasets import fetch_california_housing
import numpy as np

In [3]:
#导入原始数据
house=fetch_california_housing()
housing=pd.DataFrame(house.data,columns=house.feature_names)
target=pd.DataFrame(house.target)
housing["target"]=target
housing.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [12]:
#如何保证生成的训练集每次都一致
#法一
def split_train_test(data, test_ratio):
 shuffled_indices = np.random.permutation(len(data))#随机生成序列
 test_set_size = int(len(data) * test_ratio)
 test_indices = shuffled_indices[:test_set_size]
 train_indices = shuffled_indices[test_set_size:]
 return data.iloc[train_indices], data.iloc[test_indices]
train_set, test_set = split_train_test(housing, 0.2)
print(len(train_set), "train +", len(test_set), "test")
test_set.head()

16512 train + 4128 test


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
9021,5.3749,16.0,7.783641,1.021108,1228.0,3.240106,34.17,-118.75,3.461
3066,2.7411,27.0,4.684211,1.058824,1346.0,4.167183,35.78,-119.25,0.598
7361,2.3175,26.0,3.813606,1.038164,8551.0,4.729535,33.97,-118.18,1.545
8031,4.2841,46.0,6.521390,1.147059,944.0,2.524064,33.84,-118.13,3.124
7976,4.2135,36.0,5.886364,1.034091,974.0,3.689394,33.87,-118.20,1.614


In [15]:
#法二
import hashlib
def test_set_check(identifier, test_ratio, hash):
 return hash(np.int64(identifier)).digest()[-1] < 256 * test_ratio
def split_train_test_by_id(data, test_ratio, id_column, hash=hashlib.md5):
 ids = data[id_column]
 in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio, hash))
 return data.loc[~in_test_set], data.loc[in_test_set]
housing_with_id = housing.reset_index() # adds an `index` column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")
test_set.head()
housing_with_id["id"] = housing["Longitude"] * 1000 + housing["Latitude"]#使用数据创造id
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")
test_set.head()

,index,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target,id
8,8,2.0804,42.0,4.294118,1.117647,1206.0,2.026891,37.84,-122.26,2.267,-122222.16
10,10,3.2031,52.0,5.477612,1.079602,910.0,2.263682,37.85,-122.26,2.815,-122222.15
11,11,3.2705,52.0,4.772480,1.024523,1504.0,2.049046,37.85,-122.26,2.418,-122222.15
12,12,3.0750,52.0,5.322650,1.012821,1098.0,2.346154,37.85,-122.26,2.135,-122222.15
13,13,2.6736,52.0,4.000000,1.097701,345.0,1.982759,37.84,-122.26,1.913,-122222.16


In [16]:
#法三
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
test_set.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
20046,1.6812,25.0,4.192201,1.022284,1392.0,3.877437,36.06,-119.01,0.47700
3024,2.5313,30.0,5.039384,1.193493,1565.0,2.679795,35.14,-119.46,0.45800
15663,3.4801,52.0,3.977155,1.185877,1310.0,1.360332,37.80,-122.44,5.00001
20484,5.7376,17.0,6.163636,1.020202,1705.0,3.444444,34.28,-118.72,2.18600
9814,3.7250,34.0,5.492991,1.028037,1063.0,2.483645,36.62,-121.93,2.78000


In [24]:
#分层抽样
#拥有者的平均收入对于预测房子的平均价格十分重要，因此，我们希望在抽样时尽可能涵盖各层次的收入
housing["income_cat"] = np.ceil(housing["MedInc"] / 1.5)#向上取整
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)#减少分层类别
housing.head()
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)#n_splits:只生成一个训练集和测试集
for train_index, test_index in split.split(housing, housing["income_cat"]):
 strat_train_set = housing.loc[train_index]
 strat_test_set = housing.loc[test_index]
print(strat_test_set["income_cat"].value_counts() / len(strat_test_set))
#去除income_cat，还原数据
for set_ in (strat_train_set, strat_test_set):
 set_.drop("income_cat", axis=1, inplace=True)
strat_test_set.head()



income_cat
3.0    0.350533
2.0    0.318798
4.0    0.176357
5.0    0.114341
1.0    0.039971
Name: count, dtype: float64


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
5241,8.2816,29.0,6.715625,1.054167,2184.0,2.275000,34.12,-118.39,5.00001
17352,5.0099,24.0,7.137809,1.084806,855.0,3.021201,34.89,-120.42,1.62500
3505,4.3839,36.0,5.283636,0.981818,808.0,2.938182,34.25,-118.45,2.04600
7777,3.2708,35.0,5.491694,1.079734,1072.0,3.561462,33.91,-118.10,1.59700
14155,4.3529,38.0,6.154723,1.000000,1495.0,2.434853,32.77,-117.07,1.84000
